In [1]:
from google.cloud import storage
import os
import pandas as pd
import numpy as np
import sqlite3
from tqdm.notebook import tqdm

In [2]:
import numpy as np

In [3]:
df = pd.read_csv('gs://data_tql/WikiSQL/wikisql_train_final.csv')

In [4]:
df.shape

(56355, 8)

In [5]:
def list_blobs(bucket_name, folder_name):
    """List all files in given COS directory."""    
    blob_names = []
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))
    for blob in blobs:
        blob_names.append(blob.name)
    return blob_names
    
def list_blobs_pd(bucket_name, folder_name):
    """List all files in given COS directory."""       
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    blob_name = []
    blob_size = []
    blob_time = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)
        blob_time.append(blob.time_created)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size, blob_time)), columns=['filePath', 'size', 'timeStamp'])    
    return blobs_df

def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from COS bucket."""
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""    
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

In [6]:
bucket_name = 'data_tql'
all_blobs = list_blobs(bucket_name, 'wikisqliteDB/train')

In [7]:
import os

In [8]:
len(df)

56355

In [9]:
df.head()

,phase,table_id,question,sel,conds,agg,sql,table_tableid
0,1,1-1000181-1,Tell me what the notes are for South Australia,5,"[[3, 0, 'SOUTH AUSTRALIA']]",0,SELECT col5 FROM table_1_1000181_1 WHERE col3...,table_1_1000181_1
1,1,1-1000181-1,What is the current series where the new serie...,4,"[[5, 0, 'New series began in June 2011']]",0,SELECT col4 FROM table_1_1000181_1 WHERE col5...,table_1_1000181_1
2,1,1-1000181-1,What is the format for South Australia?,2,"[[0, 0, 'South Australia']]",0,SELECT col2 FROM table_1_1000181_1 WHERE col0...,table_1_1000181_1
3,1,1-1000181-1,Name the background colour for the Australian ...,1,"[[0, 0, 'Australian Capital Territory']]",0,SELECT col1 FROM table_1_1000181_1 WHERE col0...,table_1_1000181_1
4,1,1-10007452-3,how many times is the fuel propulsion is cng?,3,"[[5, 0, 'CNG']]",3,SELECT COUNT col3 FROM table_1_10007452_3 WHER...,table_1_10007452_3


In [10]:
BUCKET_NAME = "data_tql"
FOLDER_NAME = "wikisqliteDB"
DB_TYPE = "train"

storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)

In [ ]:
blob = bucket.blob(f"{FOLDER_NAME}/{DB_TYPE}/{filename}.sqlite")
blob.download_to_filename(filename+".sqlite")

In [31]:
for i,j in df[50:].iterrows():
    filename = str(j.table_tableid)
    blob = bucket.blob(f"{FOLDER_NAME}/{DB_TYPE}/{filename}.sqlite")
    blob.download_to_filename(filename+".sqlite")
    conn = sqlite3.connect(f'{filename}.sqlite')
    cursor = conn.cursor()
    result = cursor.execute(j.sql)
    print(result)
    break

In [32]:
print(result.fetchall())

[]


In [33]:
conn.close()

In [ ]:
all_blobs = list_blobs(bucket_name, 'wikisqliteDB/train')
failed_queries = {}
if not no_db_file:
    
    for schema in tqdm(sqlite_dict.keys()):
        filePath = sqlite_dict[schema][0]
        fileName = sqlite_dict[schema][1]
        download_blob(bucket_name, filePath, fileName)
        
        if f'sqliteDB/{fileName}' not in all_blobs:
            upload_blob(bucket_name, fileName, f'sqliteDB/{fileName}')
            
        schema_queries = query_data_blobs[query_data_blobs['db_id'] == schema]        
        if len(schema_queries) != 0:
            
            conn = sqlite3.connect(f'{schema}.sqlite')
            cursor = conn.cursor()
            schema_queries = schema_queries[['SQL']].drop_duplicates().values
            
            for schema_query in schema_queries:
                number_of_same_queries = len(query_data_blobs[(query_data_blobs['db_id'] == schema) & (query_data_blobs['SQL'] == schema_query[0])])
                try:
                    res_dict = pd.read_sql_query(schema_query[0], conn).to_dict()
                    query_data_blobs.loc[(query_data_blobs['db_id'] == schema) &
                                         (query_data_blobs['SQL'] == schema_query[0]), 'result'] = [res_dict] * number_of_same_queries
                    
                except Exception as err:
                    failed_queries[schema] = [schema_query[0], err]
            conn.close()
        os.remove(fileName)